In [37]:
import segno
import uuid
import os.path
import httplib2

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.service_account import ServiceAccountCredentials


from PIL import Image, ImageDraw, ImageFilter, ImageFont

from pypdf import PdfMerger

In [38]:
class Ticket():
    def __init__(self, uuid, seatID, bookingID, name):
        self.uuid = uuid
        self.seatID = seatID
        self.bookingID = bookingID
        self.name = name

    def __str__(self):
        rtn = ""
        rtn += self.uuid + ", "
        rtn += self.seatID + ", "
        rtn += self.bookingID  + ", "
        rtn += self.name
        return rtn


tickets = {} #tickets would be a dictionary with key of TicketID (eg T001), and value of the Ticket object (see Ticket class)

In [39]:
def loadAllDataFromSheets():
    '''This function loads data into the variable "tickets"'''
    SCOPE = ["https://www.googleapis.com/auth/spreadsheets"]

    creds = Credentials.from_authorized_user_file("token.json", SCOPE)
    service = build('sheets','v4', credentials=creds)
    #spreadsheetId = '1cJy3DYsIVj5pbszMqrekUNod-PeCrM7OCNKrhstQsnY'
    spreadsheetId = '1k9nABcbacmfZ0LiJk-q-dDhKOpDF0PAtMCU22adD57Y'

    rangeName = 'Tickets!A2:E379' #input the suitable column+row range to read data from
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheetId, range=rangeName).execute()
    values = result.get('values', [])

    #Is in order: TicketID, uuid, seatID, bookingID, name
    for val in values:
        obj = Ticket(val[1], val[2], val[3], val[4])
        tickets[val[0]] = obj

    for tic in tickets:
        print(tic+",", tickets[tic])


In [40]:
def makeQR(text): #text has no prefix
    qrcode = segno.make_qr("TICKET"+text)
    qrcode.save(f"QRPNGs/{text}.png", scale=15, light="deedff")

In [41]:
def pasteInfoOntoTicket(ticketID, ticketObj): 
    #QR code
    qr = Image.open("QRPNGs/" + ticketObj.uuid + ".png")
    qr = qr.resize((550,550))
    ticket = Image.open('template.png')
    newTicket = ticket.copy()
    newTicket.paste(qr, (70, 54))

    #Ticket ID (eg T001)
    draw = ImageDraw.Draw(newTicket)
    font = ImageFont.truetype("ArimaMadurai-Bold.ttf", 18)
    draw.text((1910, 600), ticketID, font=font, fill=(32,48,152), anchor="mm")

    #Seat No.
    font = ImageFont.truetype("ArimaMadurai-Ita.ttf", 40)
    draw.text((1730, 120), "Seat No.:", font=font, fill=(32,48,152), anchor="mm")
    font = ImageFont.truetype("ArimaMadurai-ExtraBold.ttf", 138)
    draw.text((1730, 270), ticketObj.seatID, font=font, fill=(32,48,152), anchor="mm")

    #Guest Name
    font = ImageFont.truetype("ArimaMadurai-Ita.ttf", 40)
    draw.text((1730, 420), "Guest Name:", font=font, fill=(32,48,152), anchor="mm")
    length = len(ticketObj.name)
    font = ImageFont.truetype("ArimaMadurai-Bold.ttf", 40)
    draw.text((1730, 500), ticketObj.name, font=font, fill=(32,48,152), anchor="mm")

    newTicket = newTicket.convert('RGB')
    newTicket.save("TicketPDFs/TICKET" + ticketObj.uuid + ".pdf", quality=100)

In [42]:
def combineTickets():
    #tickets is an array of ticket ids without the .pdf extension or "TICKET" prefix

    exists = []

    for t in tickets:
        merger = PdfMerger()
        if tickets[t].bookingID in exists:
            merger.append("Tickets/Tickets-"+tickets[t].bookingID+".pdf")
        else:
            exists += [tickets[t].bookingID]
        merger.append("TICKETPDFs/TICKET" + tickets[t].uuid + ".pdf")
        merger.write("Tickets/Tickets-"+tickets[t].bookingID+".pdf")
        merger.close()

In [43]:
def main():
    loadAllDataFromSheets()
    #make QRs of all tickets, order of doing so does not matter
    for tic in tickets:
        makeQR(tickets[tic].uuid)
    #create indivual tickets with relevant info pasted on
    for tic in tickets:
        pasteInfoOntoTicket(tic, tickets[tic])
    combineTickets()
    
main()

T001, db0570be-bd86-4acd-bc52-f6ae4d0c6632, H18, 78, Soh Sze Han
T002, 86efa49a-ad03-40ef-9713-c1244417e571, H19, 78, Ah Lang
T003, 0e952bac-1a6f-4268-acfa-e763187c8c3f, H20, 78, Cheng Kim
T004, 32f5c464-ce40-4ca0-b650-39325490c9b7, H21, 78, Joyce
T005, af4d21bf-3b50-4e36-a05d-76b35454c536, H22, 78, Joanne
T006, ebc9c79f-fe21-4a53-96c6-8278d612ea2f, H23, 78, Jennifer
T007, 1141a2a3-6fa5-4382-a729-567581c375e2, H24, 78, Fiona
T008, a5fb52aa-3c9d-42b3-b81d-25266fe6fc52, E7, 5, Cheng Kai Sen
T009, 60f572d8-fc3a-4d3d-8cb7-f7268987539d, E8, 5, Cheng Xin Ern
T010, f325a76b-ca26-4040-82f3-dc47db6e0027, E9, 5, Cheng Tein Lon
T011, 5267d634-e443-4994-b937-320691155fba, E10, 5, Angeline Teo
T012, 3b9ade4b-e451-4c51-8b2e-84f86f8947eb, E11, 5, Lim Tong
T013, 3e05d98f-8c4c-41ea-a132-bcedb85462be, F11, 2, Doreen Tan
T014, 052247de-6031-45d5-811e-ed117380506c, F12, 2, Anthony Ng
T015, 413ca4ea-6d41-43c2-9e27-d41137c82558, F13, 2, Joel Ng
T016, 2f3b7cb1-e066-4c4c-8430-5d46fff2203e, F14, 2, Amanda
T017